In [ ]:
import pandas as pd
import os

In [2]:
csv_path = r"C:\Users\lenka\OneDrive\Documenten\Afstuderen Master\pipeline\resultaten.csv"
ground_truth_folder = r"C:\Users\lenka\OneDrive\Documenten\Afstuderen Master\detectie model\UFPR-AMR Dataset\testing"

In [3]:
df = pd.read_csv(csv_path, skiprows=2, header=None, names=["bestand", "voorspelling1", "voorspelling2"])

# Ground truth toevoegen

In [4]:
# Functie om alleen de reading te extraheren
def extract_reading(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip().startswith("reading:"):
                return line.split("reading:")[1].strip()
    return None

# Voeg voor elke bestand de reading toe aan nieuwe kolom 'ground_truth'
# Filter ongeldige regels
df = df[~df["bestand"].str.contains("GEEN OBJECTEN GEDETECTEERD", na=False)]

# Voeg ground truth toe
df["ground_truth"] = df["bestand"].apply(

    lambda name: extract_reading(os.path.join(ground_truth_folder, os.path.splitext(name)[0] + ".txt"))
)

# Opslaan naar nieuw CSV-bestand
df.to_csv("resultaten_met_ground_truth.csv", index=False)

In [8]:
import pandas as pd

# Laad het CSV-bestand, alles als string om .0-problemen te vermijden
df = pd.read_csv("resultaten_met_ground_truth.csv", dtype=str)

# Zorg dat voorspellingen altijd 5 cijfers zijn met voorloopnullen
for col in ["voorspelling1", "voorspelling2"]:
    df[col] = df[col].apply(
        lambda x: str(x).split(".")[0].zfill(5) if pd.notnull(x) else x
    )

df.to_csv("resultaten_gecorrigeerd.csv", index=False)

In [9]:
df = pd.read_csv("resultaten_gecorrigeerd.csv", dtype=str)
df = df.reset_index(drop=True)

# === Kolomnamen corrigeren ===
# Hernoem kolommen tijdelijk voor gemak
df = df.rename(columns={
    "voorspelling1": "zonder_reconstructie",
    "voorspelling2": "met_reconstructie"
})

# Resultaten

In [ ]:
for kolom in ["ground_truth", "zonder_reconstructie", "met_reconstructie"]:
    df[kolom] = df[kolom].astype(str).str.strip().str.zfill(5)  # pas aan als het 6 moet zijn

df["match_zonder"] = df["zonder_reconstructie"] == df["ground_truth"]
df["match_met"] = df["met_reconstructie"] == df["ground_truth"]

def totale_cijfer_accuracy(grond, voorspelling):
    totaal_cijfers = 0
    correcte_cijfers = 0
    for gt, pred in zip(grond, voorspelling):
        gt = str(gt).strip()
        pred = str(pred).strip()
        lengte = min(len(gt), len(pred))
        totaal_cijfers += lengte
        correcte_cijfers += sum(g == p for g, p in zip(gt, pred))
    if totaal_cijfers == 0:
        return 0.0
    return correcte_cijfers / totaal_cijfers

totaal = len(df)
aantal_goed_zonder = df["match_zonder"].sum()
aantal_goed_met = df["match_met"].sum()
cijfer_acc_zonder = totale_cijfer_accuracy(df["ground_truth"], df["zonder_reconstructie"])
cijfer_acc_met = totale_cijfer_accuracy(df["ground_truth"], df["met_reconstructie"])

hersteld = df[(~df["match_zonder"]) & (df["match_met"])]
verpest = df[(df["match_zonder"]) & (~df["match_met"])]

print("ZONDER reconstructie:")
print(f"  Aantal goed (volledige match): {aantal_goed_zonder}/{totaal}")
print(f"  Accuracy volledige reeks: {aantal_goed_zonder / totaal:.2%}")
print(f"  Totale cijfer-accuracy: {cijfer_acc_zonder:.2%}")

print("\nMET reconstructie:")
print(f"  Aantal goed (volledige match): {aantal_goed_met}/{totaal}")
print(f"  Accuracy volledige reeks: {aantal_goed_met / totaal:.2%}")
print(f"  Totale cijfer-accuracy: {cijfer_acc_met:.2%}")

print("\nHersteld door reconstructie (was fout, is nu goed):")
print(hersteld[["zonder_reconstructie", "met_reconstructie", "ground_truth"]])

print("\nVerpest door reconstructie (was goed, is nu fout):")
print(verpest[["zonder_reconstructie", "met_reconstructie", "ground_truth"]])

ZONDER reconstructie:
  Aantal goed (volledige match): 175/650
  Accuracy volledige reeks: 26.92%
  Totale cijfer-accuracy: 60.98%

MET reconstructie:
  Aantal goed (volledige match): 165/650
  Accuracy volledige reeks: 25.38%
  Totale cijfer-accuracy: 61.17%

Hersteld door reconstructie (was fout, is nu goed):
    zonder_reconstructie met_reconstructie ground_truth
12                 07000             00000        00000
16                 05270             05290        05290
20                 99389             09389        09389
32                 97453             07453        07453
107                44950             24950        24950
155                14641             04641        04641
159                20389             20380        20380
182                72823             72822        72822
208                70279             70276        70276
211                09227             09221        09221
270                96803             56803        56803
276            

In [ ]:
df = pd.read_csv("resultaten_gecorrigeerd.csv", dtype=str)
df = df.reset_index(drop=True)

df = df.rename(columns={
    "voorspelling1": "zonder_reconstructie",
    "voorspelling2": "met_reconstructie"
})

for kolom in ["ground_truth", "zonder_reconstructie", "met_reconstructie"]:
    df[kolom] = df[kolom].astype(str).str.strip().str.zfill(5)

df["match_zonder"] = df["zonder_reconstructie"] == df["ground_truth"]
df["match_met"] = df["met_reconstructie"] == df["ground_truth"]

def totale_cijfer_accuracy(grond, voorspelling):
    totaal_cijfers = 0
    correcte_cijfers = 0
    for gt, pred in zip(grond, voorspelling):
        lengte = min(len(gt), len(pred))
        totaal_cijfers += lengte
        correcte_cijfers += sum(g == p for g, p in zip(gt, pred))
    return correcte_cijfers / totaal_cijfers if totaal_cijfers > 0 else 0.0

totaal = len(df)
aantal_goed_zonder = df["match_zonder"].sum()
aantal_goed_met = df["match_met"].sum()
cijfer_acc_zonder = totale_cijfer_accuracy(df["ground_truth"], df["zonder_reconstructie"])
cijfer_acc_met = totale_cijfer_accuracy(df["ground_truth"], df["met_reconstructie"])

hersteld = (~df["match_zonder"]) & (df["match_met"])
verpest = (df["match_zonder"]) & (~df["match_met"])

aantal_hersteld = hersteld.sum()
aantal_verpest = verpest.sum()

# Resultaten
print("ZONDER reconstructie:")
print(f"  Volledige match: {aantal_goed_zonder}/{totaal}")
print(f"  Reeks accuracy: {aantal_goed_zonder / totaal:.2%}")
print(f"  Cijfer accuracy: {cijfer_acc_zonder:.2%}")

print("\nMET reconstructie:")
print(f"  Volledige match: {aantal_goed_met}/{totaal}")
print(f"  Reeks accuracy: {aantal_goed_met / totaal:.2%}")
print(f"  Cijfer accuracy: {cijfer_acc_met:.2%}")

print("\nReconstructie-effect:")
print(f"  Hersteld: {aantal_hersteld}")
print(f"  Verpest: {aantal_verpest}")


ZONDER reconstructie:
  Volledige match: 175/650
  Reeks accuracy: 26.92%
  Cijfer accuracy: 60.98%

MET reconstructie:
  Volledige match: 165/650
  Reeks accuracy: 25.38%
  Cijfer accuracy: 61.17%

Reconstructie-effect:
  Hersteld: 20
  Verpest: 30
